In [1]:
from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp
import pyomo.environ as pyo
from RPB_model import *
import pickle
from pyomo.contrib.doe import DesignOfExperiments, MeasurementVariables, DesignVariables

### Model create

In [2]:
def create_model(model_option = None):
    """ This model returns the rotary bed model
    """

    RPB = ConcreteModel()
    
    parameter = {
    "DmCO2": 5.3e-5, 
    "C1": 2.562434e-12, 
    "hgx": 25 * 1e-3, 
    "Cp_sol": 1.457, 
    "rho_sol": 1144, 
    "delH_1": 98.76,
    "delH_2": 77.11, 
    "delH_3": 21.25
}

    RPB.ads = RPB_model(mode="adsorption", kaug=False, param_perturb=parameter)
    RPB.des = RPB_model(mode="desorption", kaug=False, param_perturb=parameter)

    # fix BCs
    RPB.ads.P_in.fix(1.1)
    RPB.ads.Tg_in.fix()
    RPB.ads.y_in.fix()
    RPB.ads.P_out.fix(1.01325)

    RPB.des.P_in.fix(1.1)
    RPB.des.Tg_in.fix()
    RPB.des.y_in.fix()
    RPB.des.P_out.fix(1.01325)

    # connect rich stream
    # unfix inlet loading and temperature to the desorption section. (No mass transfer at boundaries so z=0 and z=1 need to remain fixed.)
    for z in RPB.des.z:
        if z!=0 and z!=1:
            RPB.des.qCO2_in[z].unfix()
            RPB.des.Ts_in[z].unfix()

    # add equality constraint equating inlet desorption loading to outlet adsorption loading. Same for temperature.
    @RPB.Constraint(RPB.des.z)
    def rich_loading_constraint(RPB,z):
        if z==0 or z==1:
            return Constraint.Skip
        else:
            return RPB.des.qCO2_in[z] == RPB.ads.qCO2[z,1]

    @RPB.Constraint(RPB.des.z)
    def rich_temp_constraint(RPB,z):
        if z==0 or z==1:
            return Constraint.Skip
        else:
            return 1e-2*RPB.des.Ts_in[z] == 1e-2*RPB.ads.Ts[z,1]


    # connect lean stream
    # unfix inlet loading to the adsorption section
    for z in RPB.ads.z:
        if z!=0 and z!=1:
            RPB.ads.qCO2_in[z].unfix()

    # add equality constraint equating inlet adsorption loading to outlet desorption loading
    @RPB.Constraint(RPB.ads.z)
    def lean_loading_constraint(RPB,z):
        if z==0 or z==1:
            return Constraint.Skip
        else:
            return RPB.des.qCO2[z,1] == RPB.ads.qCO2_in[z]


    # create Block init object
    init_obj = BlockTriangularizationInitializer()

    init_obj.config.block_solver_call_options = {"tee": True}
    # init_obj.config.block_solver_options = {"halt_on_ampl_error": "yes"}

    # turn on solids mass transfer (with the loadings connected at the rich and lean ends, solids mass transfer has to be turned on or no solution exists)
    RPB.ads.R_MT_solid = 1
    RPB.des.R_MT_solid = 1

    # now turn on the other equations using the homotopy routine

    variables_list = [
        RPB.ads.R_MT_gas,
        RPB.des.R_MT_gas,
        RPB.ads.R_MT_coeff,
        RPB.des.R_MT_coeff,
        RPB.ads.R_HT_ghx,
        RPB.des.R_HT_ghx,
        RPB.ads.R_HT_gs,
        RPB.des.R_HT_gs,
        RPB.ads.R_delH,
        RPB.des.R_delH,
    ]

    targets_list = [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
    ]


    RPB.Obj = pyo.Objective(expr=0, sense=pyo.minimize)
    
    return RPB


### Parameter, Design variables, Measurement variables

In [3]:
parameter_dict = {
     "ads.C1": 2.562434e-12, 
     "des.C1": 2.562434e-12,
     "ads.hgx": 25 * 1e-3, 
      "des.hgx": 25 * 1e-3,
     "ads.delH_1": 98.76, 
     "des.delH_1": 98.76,
     "ads.delH_2": 77.11, 
    "des.delH_2": 77.11,
     "ads.delH_3": 21.25, 
    "des.delH_3": 21.25
}

In [4]:
# measurements 
measure_class = MeasurementVariables()
variable_names = ['ads.C[N2,0.07125,0.255]', 
                 'ads.C[N2,0.091667,0.993333]', 
                 'ads.C[CO2,0,0.5]', 
                 'ads.y[CO2,0,0.993333]', 
                 'ads.y[CO2,0.07125,0.993333]', 
                 'ads.qCO2[0.214167,0.336667]', 
                 'ads.qCO2[0.255,0.255]', 
                 'ads.qCO2[0.214167,0.5]', 
                 'ads.qCO2[0.3775,0.01]', 
                   'ads.Cs_r[0.6225,0.01]',
                  'ads.Cs_r[0.469375,0.091667]', 
                  'ads.Cs_r[0.19375,0.5]', 
                  'des.C[N2,0.326458,0.003333]', 
                  'des.C[N2,0.43875,0.255]', 
                  'des.y[N2,0.520417,1]', 
                  'des.C[N2,0.745,0.993333]', 
                   'des.y[CO2,0.3775,0.99]',
                  'des.y[CO2,0.43875,0.01]', 
                  'des.y[CO2,0.68375,0]', 
                  'des.C[CO2,0.949167,0]',
                   'des.C[CO2,0.99,0.01]', 
                  'des.y[CO2,0.993333,0.003333]'
                  
                 ]
measure_class.set_variable_name_list(variable_names)

print(measure_class.variable_names)

['ads.C[N2,0.07125,0.255]', 'ads.C[N2,0.091667,0.993333]', 'ads.C[CO2,0,0.5]', 'ads.y[CO2,0,0.993333]', 'ads.y[CO2,0.07125,0.993333]', 'ads.qCO2[0.214167,0.336667]', 'ads.qCO2[0.255,0.255]', 'ads.qCO2[0.214167,0.5]', 'ads.qCO2[0.3775,0.01]', 'ads.Cs_r[0.6225,0.01]', 'ads.Cs_r[0.469375,0.091667]', 'ads.Cs_r[0.19375,0.5]', 'des.C[N2,0.326458,0.003333]', 'des.C[N2,0.43875,0.255]', 'des.y[N2,0.520417,1]', 'des.C[N2,0.745,0.993333]', 'des.y[CO2,0.3775,0.99]', 'des.y[CO2,0.43875,0.01]', 'des.y[CO2,0.68375,0]', 'des.C[CO2,0.949167,0]', 'des.C[CO2,0.99,0.01]', 'des.y[CO2,0.993333,0.003333]']


In [5]:
design_gen = DesignVariables()

design_gen.set_variable_name_list(['ads.P_in', 'des.P_in'])

print(design_gen.variable_names)

['ads.P_in', 'des.P_in']


## Analyze with k_aug

In [6]:
# choose from 'sequential_finite', 'direct_kaug'
sensi_opt = "direct_kaug"
#sensi_opt = "sequential_finite"

# Define experiments
design_names = design_gen.variable_names
exp1 = [1.1, 1.1]
exp1_design_dict = dict(zip(design_names, exp1))

design_gen.update_values(exp1_design_dict)

In [7]:
doe_object = DesignOfExperiments(parameter_dict, design_gen,
                                 measure_class, create_model)


result = doe_object.compute_FIM(mode=sensi_opt, FIM_store_name = 'dynamic.csv', 
                                read_output=None,
                                scale_nominal_param_value=True,
                                formula = "central")


result.result_analysis()

current version
Ipopt 3.13.2: output_file=/tmp/tmpmsn2vnyfipopt_out
max_iter=100
max_cpu_time=60


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a coll

   1  0.0000000e+00 8.30e+01 6.62e+02  -1.0 1.23e+03    -  4.27e-01 9.91e-01H  1
   2  0.0000000e+00 3.77e+01 6.41e+03  -1.0 2.97e+02    -  2.79e-01 1.00e+00h  1
   3  0.0000000e+00 8.33e+01 2.01e+03  -1.0 3.91e+02    -  8.26e-01 1.00e+00H  1
   4  0.0000000e+00 4.95e+01 4.37e+03  -1.0 2.22e+02    -  6.26e-01 1.00e+00H  1
   5  0.0000000e+00 4.56e+01 2.34e+03  -1.0 1.60e+02    -  4.29e-01 1.00e+00f  1
   6  0.0000000e+00 9.11e+01 5.40e+03  -1.0 2.84e+02    -  8.42e-01 1.00e+00h  1
   7  0.0000000e+00 4.26e+01 8.04e+03  -1.0 3.02e+02    -  7.15e-01 1.00e+00h  1
   8  0.0000000e+00 4.40e+01 4.82e+03  -1.0 2.31e+02    -  1.00e+00 1.00e+00h  1
   9  0.0000000e+00 4.04e+01 1.76e+03  -1.0 1.74e+02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  0.0000000e+00 3.25e+01 2.14e+03  -1.0 1.12e+02    -  1.00e+00 1.00e+00h  1
  11  0.0000000e+00 1.74e+01 5.40e+02  -1.0 6.94e+01    -  1.00e+00 1.00e+00h  1
  12  0.0000000e+00 3.53e+00

2023-11-17 11:40:01 [INFO] idaes.core.solvers.homotopy: Homotopy Iteration 4. Next Step: 0.3588541666666667 (Current: 0.3041666666666667)
Ipopt 3.13.2: output_file=/tmp/tmps1oz7uo1ipopt_out
max_iter=100
max_cpu_time=60


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity mat

2023-11-17 11:40:26 [INFO] idaes.core.solvers.homotopy: Homotopy Iteration 6. Next Step: 0.4424045138888889 (Current: 0.4044270833333334)
Ipopt 3.13.2: output_file=/tmp/tmpzutr_o_kipopt_out
max_iter=100
max_cpu_time=60


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity mat

  26  0.0000000e+00 8.78e+00 8.79e+02  -1.7 2.42e+01    -  1.00e+00 5.00e-01h  2
  27  0.0000000e+00 4.45e-02 7.74e+00  -1.7 1.52e+00    -  1.00e+00 1.00e+00h  1
  28  0.0000000e+00 5.43e-06 1.00e-03  -3.8 1.69e-02    -  1.00e+00 1.00e+00h  1
  29  0.0000000e+00 1.75e-09 2.14e-09  -5.7 3.64e-06    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 29

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   2.9692993161966402e-11    1.7478569702689126e-09
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   2.9692993161966402e-11    1.7478569702689126e-09


Number of objective function evaluations             = 52
Number of objective gradient evaluations             = 30
Number of equality constraint evaluations            = 52
Number of ine

...

2023-11-17 11:42:26 [INFO] idaes.core.solvers.homotopy: Homotopy Iteration 12. Next Step: 0.5986222190418462 (Current: 0.5587022101742097)
Ipopt 3.13.2: output_file=/tmp/tmpaklvqsb8ipopt_out
max_iter=100
max_cpu_time=60


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity ma

Ipopt 3.13.2: output_file=/tmp/tmp4h0ff0moipopt_out
max_iter=100
max_cpu_time=60


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortra

   1  0.0000000e+00 2.34e+02 4.24e+01  -1.0 4.74e+02    -  8.66e-01 9.91e-01h  1
   2  0.0000000e+00 1.05e+02 1.89e+02  -1.0 4.18e+02    -  8.11e-01 1.00e+00H  1
   3  0.0000000e+00 2.56e+02 3.66e+02  -1.0 5.25e+02    -  9.60e-01 1.00e+00h  1
   4  0.0000000e+00 1.96e+02 8.20e+00  -1.0 7.86e+02    -  1.00e+00 1.00e+00H  1
   5  0.0000000e+00 2.56e+02 5.09e+00  -1.0 6.50e+02    -  1.00e+00 1.00e+00h  1
   6  0.0000000e+00 1.49e+02 5.70e+00  -1.0 6.77e+02    -  1.00e+00 5.00e-01h  2
   7  0.0000000e+00 2.57e+02 3.74e+00  -1.0 2.85e+02    -  1.00e+00 1.00e+00h  1
   8  0.0000000e+00 1.00e+02 4.42e+00  -1.0 5.52e+02    -  1.00e+00 1.00e+00H  1
   9  0.0000000e+00 2.57e+02 3.85e+00  -1.0 5.01e+02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  0.0000000e+00 1.44e+02 6.57e+00  -1.0 7.32e+02    -  1.00e+00 5.00e-01h  2
  11  0.0000000e+00 2.57e+02 3.58e+00  -1.0 2.22e+02    -  1.00e+00 1.00e+00h  1
  12  0.0000000e+00 9.59e+01

   1  0.0000000e+00 1.62e+02 3.05e+01  -1.0 3.37e+02    -  9.06e-01 9.91e-01h  1
   2  0.0000000e+00 1.57e+01 1.27e+01  -1.0 1.98e+02    -  9.92e-01 1.00e+00h  1
   3  0.0000000e+00 4.27e-03 9.85e-02  -1.7 1.39e+01    -  1.00e+00 1.00e+00h  1
   4  0.0000000e+00 1.93e-07 1.47e-04  -3.8 7.69e-03    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 4

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   9.6739128929712592e-09    1.9348679325403850e-07
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   9.6739128929712592e-09    1.9348679325403850e-07


Number of objective function evaluations             = 5
Number of objective gradient evaluations             = 5
Number of equality constraint evaluations            = 5
Number of inequal

component keys that are not exported as part of the NL file.  Skipping.
component keys that are not exported as part of the NL file.  Skipping.
INFO: elapsed time: 359.5


In [10]:
# Results summary
print("======Results Summary======")
print("Four design criteria log10() value:")
print("A-optimality:", np.log10(result.trace))
print("D-optimality:", np.log10(result.det))
print("E-optimality:", np.log10(result.min_eig))
print("Modified E-optimality:", np.log10(result.cond))
print(result.FIM)
print(np.linalg.eig(result.FIM))

======Results Summary======
Four design criteria log10() value:
A-optimality: 1.956100353692046
D-optimality: -33.6191816764711
E-optimality: -21.792045386289722
Modified E-optimality: 23.708257208855542
[[ 1.27413564e-01  2.56317693e-01 -2.96022973e-01  3.50832504e-02
   1.45833208e+00 -1.95645964e-01 -1.18959366e+00 -7.28840396e-03
   1.53493823e-02 -1.15208535e-13]
 [ 2.56317693e-01  8.69064120e-01 -1.38130383e+00 -7.83207299e-02
   4.23547358e+00 -9.60512915e-01 -3.54341774e+00  5.86405303e-02
   7.18157884e-02  1.54374312e-11]
 [-2.96022973e-01 -1.38130383e+00  6.06826888e+00  9.10384461e-01
  -3.26049568e+00  2.74283642e+00  3.01130625e+00 -5.59683269e-01
  -1.60526905e-01 -8.61177315e-11]
 [ 3.50832504e-02 -7.83207299e-02  9.10384461e-01  2.85386867e-01
  -3.64342099e-01  4.33548893e-01  3.48824854e-01 -1.33956737e-01
  -2.26456002e-02 -1.91569812e-11]
 [ 1.45833208e+00  4.23547358e+00 -3.26049568e+00 -3.64342099e-01
   4.92088388e+01 -4.23648434e+00 -3.97537898e+01  1.56313073e